iDT1278 sbml model is not valid and matlab model has no annotations information will add annotation from most recent ecoli model. 

Based and modified from https://github.com/ChristianLieven/Yarrowia_lipolytica_W29-GEM/


In [1]:
from cobra.io import read_sbml_model, load_matlab_model, write_sbml_model

In [2]:
Av_model = load_matlab_model("../Data/Models/iDT1278.mat")
Ec_model = read_sbml_model("../Data/Models/iAF1260.xml")
Kp_model = read_sbml_model("../Data/Models/iYL1228.xml")
Gm_model = read_sbml_model("../Data/Models/iAF987.xml")

C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\cobra\core\reaction.py:464: UserWarning: malformed gene_reaction_rule '((Avin_11760) and (Avin_13240) and (Avin_02950)) or ((Avin_45310) or (Avin_42560)' for <Reaction MPTG at 0x2d6da409a88>
  warn("malformed gene_reaction_rule '%s' for %s" % (new_rule, repr(self)))
C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\cobra\core\reaction.py:464: UserWarning: malformed gene_reaction_rule '((Avin_11760) and (Avin_13240) and (Avin_02950)) or ((Avin_45310) or (Avin_42560)' for <Reaction MPTG2 at 0x2d6da41dbc8>
  warn("malformed gene_reaction_rule '%s' for %s" % (new_rule, repr(self)))


In [3]:
#Fix malformed MPTG gene
MPTG = Av_model.reactions.get_by_id('MPTG')
MPTG.gene_reaction_rule = '(Avin_11760) and (Avin_13240)'

MPTG2 = Av_model.reactions.get_by_id('MPTG2')
MPTG2.gene_reaction_rule = '(Avin_45310) or (Avin_42560)'

In [4]:
#rewrite matlab model to SBML
write_sbml_model(Av_model, "../Data/Models/iDT1278_n.xml")

In [5]:
#Determine how many metabolites have seed annotation can choose any annotation 

rxn_count = 0
for met in Av_model.metabolites:
    if "seed.compound" not in met.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.metabolites))
        

2003 2003


There are no annotations at all for any metabolite in the model

In [6]:
#Compare annotation of Av model to Ec model
Av_model.metabolites.ac_c.annotation

{}

In [7]:
Ec_model.metabolites.ac_c.annotation

{'sbo': 'SBO:0000247',
 'bigg.metabolite': 'ac',
 'biocyc': 'META:ACET',
 'chebi': ['CHEBI:13704',
  'CHEBI:22165',
  'CHEBI:22169',
  'CHEBI:40480',
  'CHEBI:15366',
  'CHEBI:2387',
  'CHEBI:30089',
  'CHEBI:40486'],
 'envipath': ['5882df9c-dae1-4d80-a40e-db4724271456/compound/9e26dcbe-4db9-46a0-8614-9f03545032d2',
  '5882df9c-dae1-4d80-a40e-db4724271456/compound/5e4989fc-13d3-45d4-ad57-3be380a9d3c0',
  '650babc9-9d68-4b73-9332-11972ca26f7b/compound/a8f0be58-24e8-441b-8d81-a516a0ead4b3',
  '4fd7f3e0-dd25-43ac-9453-dda3e52396e4/compound/d45256fe-61fa-4f5b-bb16-91a3d615e3d8',
  '32de3cf4-e3e6-4168-956e-32fa5ddb0ce1/compound/b545cabc-8c9e-4b20-8848-efa015b481ea',
  '650babc9-9d68-4b73-9332-11972ca26f7b/compound/3e2d750f-df31-4445-9255-163c627e9b4a'],
 'hmdb': 'HMDB00042',
 'inchi_key': 'QTBSBXVTEAMEQO-UHFFFAOYSA-M',
 'kegg.compound': 'C00033',
 'kegg.drug': 'D00010',
 'lipidmaps': 'LMFA01010002',
 'metanetx.chemical': 'MNXM26',
 'reactome.compound': ['1524044', '390305', '113539', '20228

_________________
Match metabolite from E. coli model and A. vinelandii model from id and compartment and rewrite Av annotation with full annotation from Ec

In [8]:
for met in Av_model.metabolites:
    for met_ec in Ec_model.metabolites:
        if met.id in met_ec.id and met.compartment == met_ec.compartment:
            met.annotation.update(met_ec.annotation)

In [9]:
#check for new annotation in Av model 
Av_model.metabolites.ac_c.annotation

{'sbo': 'SBO:0000247',
 'bigg.metabolite': 'sulfac',
 'biocyc': 'META:CPD-10246',
 'chebi': ['CHEBI:34987', 'CHEBI:49876', 'CHEBI:58824', 'CHEBI:50519'],
 'envipath': ['650babc9-9d68-4b73-9332-11972ca26f7b/compound/ed8ac75f-3a91-484c-9d21-d318ef0efef6',
  '650babc9-9d68-4b73-9332-11972ca26f7b/compound/ca85f0e1-e6b7-458b-8f88-d4ed6b580217'],
 'hmdb': 'HMDB01488',
 'inchi_key': 'AGGIJOLULBJGTQ-UHFFFAOYSA-L',
 'kegg.compound': 'C14179',
 'kegg.drug': 'D00049',
 'lipidmaps': 'LMFA01060003',
 'metanetx.chemical': 'MNXM1949',
 'reactome.compound': ['8869604', '197230'],
 'sabiork': '1994',
 'seed.compound': 'cpd09878',
 'slm': '000000449'}

In [10]:
#Test again for number of annotations
rxn_count = 0
for met in Av_model.metabolites:
    if "seed.compound" not in met.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.metabolites))


541 2003


In [11]:
#not all metabolites where annotated from ecoli
Av_model.metabolites.fdxr_42_c.annotation

{}

Still 541 metabolites that are not annotated, These metabolites could be from other models the iDT1278 model was built on. 

Will run with Geobacter model iAF987 to introduce missing annotations then rerun E. coli to keep that as the main source

In [12]:
for met in Av_model.metabolites:
    for met_gm in Gm_model.metabolites:
        if met.id in met_gm.id and met.compartment == met_gm.compartment:
            met.annotation.update(met_gm.annotation)

In [13]:

Av_model.metabolites.fdxr_42_c.annotation

{'sbo': 'SBO:0000247',
 'bigg.metabolite': 'fdxr_42',
 'metanetx.chemical': 'MNXM145542'}

In [14]:
#Test again for number of annotations
rxn_count = 0
for met in Av_model.metabolites:
    if "metanetx.chemical" not in met.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.metabolites))

282 2003


Got it down to 282 missing annotations will replace with E. coli just in case it overwrote the orginal annotation 

In [15]:
for met in Av_model.metabolites:
    for met_ec in Ec_model.metabolites:
        if met.id in met_ec.id and met.compartment == met_ec.compartment:
            met.annotation.update(met_ec.annotation)

In [16]:
#run same procedure for reactions
rxn_count = 0
for rxn in Av_model.reactions:
    if "seed.reaction" not in rxn.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.reactions))


2469 2469


In [17]:
#if reaction ids match than write annotation
for rxn in Av_model.reactions:
    for rxn_ec in Ec_model.reactions:
        if rxn.id == rxn_ec.id:
            rxn.annotation.update(rxn_ec.annotation)



In [18]:
#check citrate synthase 
Av_model.reactions.CS.annotation

{'sbo': 'SBO:0000176',
 'bigg.reaction': 'CS',
 'biocyc': ['META:CITSYN-RXN', 'META:RXN-14905'],
 'ec-code': ['2.3.3.16', '2.3.3.1', '2.3.3.3'],
 'kegg.reaction': 'R00351',
 'metanetx.reaction': 'MNXR96920',
 'reactome.reaction': ['R-GGA-373006',
  'R-GGA-70975',
  'R-CEL-70975',
  'R-HSA-70975',
  'R-DRE-70975',
  'R-MMU-70975',
  'R-ATH-70975',
  'R-SPO-70975',
  'R-OSA-70975',
  'R-SCE-70975',
  'R-SSC-70975',
  'R-DDI-70975',
  'R-RNO-70975',
  'R-DME-70975',
  'R-PFA-70975',
  'R-XTR-70975',
  'R-CFA-70975',
  'R-BTA-70975'],
 'rhea': ['16847', '16846', '16845', '16848'],
 'sabiork': '267',
 'seed.reaction': 'rxn00256'}

In [19]:
#recheck reaction annotation count
rxn_count = 0
for rxn in Av_model.reactions:
    if "seed.reaction" not in rxn.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.reactions))

969 2469


969 reactions lack annotaiton 

In [20]:
#rerun with GM model
#if reaction ids match than write annotation
for rxn in Av_model.reactions:
    for rxn_gm in Gm_model.reactions:
        if rxn.id == rxn_ec.id:
            rxn.annotation.update(rxn_gm.annotation)

In [21]:
#recheck reaction annotation count
rxn_count = 0
for rxn in Av_model.reactions:
    if "seed.reaction" not in rxn.annotation:
        rxn_count +=1
        
print(rxn_count, len(Av_model.reactions))

969 2469


In [22]:
write_sbml_model(Av_model, "./Models/Av_annotate.xml")